In [4]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import time

In [18]:
amuse = pd.read_csv('data/pj1_amusementpark_01.csv', encoding='UTF-8', index_col=0)

In [19]:
amuse

,도로명전체주소,사업장명,분류
0,대구광역시 달성군 가창면 가창로 891,주식회사 스파밸리,테마파크
1,경상북도 영주시 회헌로 434 (아지동),판타시온 리조트,테마파크
2,"경기도 의정부시 장곡로 22, 아일랜드캐슬 A동 (장암동)",아일랜드캐슬,테마파크
3,강원특별자치도 평창군 봉평면 태기로 174 (주)휘닉스평창,휘닉스 블루캐니언,테마파크
4,경상남도 양산시 하북면 통도7길 68,(주)동일리조트 통도환타지아,테마파크
...,...,...,...
73,충청남도 보령시 대해로 876 (신흑동),레그랜드펀비치 호텔앤워터파크,테마파크
74,NaN,주식회사 가고파랜드,테마파크
75,부산광역시 영도구 해양로 435-1 (동삼동),월드카니발 부산,테마파크
76,서울특별시 광진구 능동로 216 (능동),어린이대공원놀이동산(주),테마파크


In [24]:
amuse1 =amuse.rename(columns={"도로명전체주소":"city","사업장명":"place_name","분류":"category"})
amuse1

,city,place_name,category
0,대구광역시 달성군 가창면 가창로 891,주식회사 스파밸리,테마파크
1,경상북도 영주시 회헌로 434 (아지동),판타시온 리조트,테마파크
2,"경기도 의정부시 장곡로 22, 아일랜드캐슬 A동 (장암동)",아일랜드캐슬,테마파크
3,강원특별자치도 평창군 봉평면 태기로 174 (주)휘닉스평창,휘닉스 블루캐니언,테마파크
4,경상남도 양산시 하북면 통도7길 68,(주)동일리조트 통도환타지아,테마파크
...,...,...,...
73,충청남도 보령시 대해로 876 (신흑동),레그랜드펀비치 호텔앤워터파크,테마파크
74,NaN,주식회사 가고파랜드,테마파크
75,부산광역시 영도구 해양로 435-1 (동삼동),월드카니발 부산,테마파크
76,서울특별시 광진구 능동로 216 (능동),어린이대공원놀이동산(주),테마파크


In [28]:
amuse1

,city,place_name,category
0,대구광역시 달성군 가창면 가창로 891,주식회사 스파밸리,테마파크
1,경상북도 영주시 회헌로 434 (아지동),판타시온 리조트,테마파크
2,"경기도 의정부시 장곡로 22, 아일랜드캐슬 A동 (장암동)",아일랜드캐슬,테마파크
3,강원특별자치도 평창군 봉평면 태기로 174 (주)휘닉스평창,휘닉스 블루캐니언,테마파크
4,경상남도 양산시 하북면 통도7길 68,(주)동일리조트 통도환타지아,테마파크
...,...,...,...
73,충청남도 보령시 대해로 876 (신흑동),레그랜드펀비치 호텔앤워터파크,테마파크
74,NaN,주식회사 가고파랜드,테마파크
75,부산광역시 영도구 해양로 435-1 (동삼동),월드카니발 부산,테마파크
76,서울특별시 광진구 능동로 216 (능동),어린이대공원놀이동산(주),테마파크


In [34]:
# csv 파일 불러오는 함수

def readCsv():
    
    # csv 불러오기
    amuse = pd.read_csv('data/pj1_amusementpark_01.csv', encoding='UTF-8', index_col=0)
    
    keyword_list=[]
    
    # 리스트로 갖고오기
    for keyword in amuse1['place_name']:
        keyword = keyword.replace(" ", "+")
        keyword_list.append(keyword)

    return keyword_list

In [35]:
readCsv()

['주식회사+스파밸리',
 '판타시온+리조트',
 '아일랜드캐슬',
 '휘닉스+블루캐니언',
 '(주)동일리조트+통도환타지아',
 '(주)동일리조트+통도환타지아',
 '롯데워터파크',
 '용마공원놀이동산',
 '(주)대청+비치랜드',
 '대천필랜드',
 '(주)인천도시관광',
 '유케이+펀페어스+리미티드+한국지점',
 '(주)드림랜드',
 '어드밴쳐팰리스',
 '(주)미월드',
 '진해파크랜드(주)',
 '썬비치+워터파크',
 '로봇랜드',
 '율포해수풀장',
 '꿈돌이랜드',
 '보문레저타운',
 '자유랜드',
 '제일흥업(주)부곡하와이실외유기장',
 '마린월드',
 '남원랜드',
 '주)스파월드',
 '(주)서울드림랜드',
 '이월드',
 '클럽디오아시스(CLUBD+OASIS)+스파&워터파크',
 '금오랜드',
 '상록랜드',
 '소노벨+천안',
 '(주)금호리조트아산스파비스',
 '호텔마스타+대천',
 '에버랜드',
 '캐리비안베이',
 '한국민속촌+가족공원',
 '9.81+PARK+JEJU',
 '경주월드주식회사+(경주지점)',
 '블루원+워터파크',
 '아쿠아필드+하남',
 '하니랜드',
 '주식회사+썬밸리',
 '유월드루지테마파크',
 '웰리힐리파크+워터플래닛',
 '웰리힐리파크+워터플래닛',
 '육림랜드',
 '레고랜드코리아',
 '용평피크아일랜드',
 '오션700',
 '(주)하이원추추파크',
 '하이원+워터월드',
 '신화테마파크',
 '신화워터파크',
 '롯데월드+어드벤처+부산',
 '(주)소노인터내셔널+오션월드',
 '한화리조트(주)+설악워터피아',
 '롯데리조트+속초+워터파크',
 '에이치제이디오션리조트+주식회사',
 '중흥골드스파&리조트',
 '테마파크+소풍',
 '대전오월드',
 '로봇랜드',
 '가야랜드',
 '오션어드벤처',
 '에이치제이디오션리조트+주식회사',
 '(주)서울랜드',
 '구미산업개발(주)+서울어린이대공원+놀이동산지점',
 '웅진플레이도시',
 '광주패밀리랜드(주)',
 '(주)롯데월드어드벤처',
 '워터킹덤',
 '

In [8]:
#2. url에 raw데이터 갖고오기

def url_raw():
    
    keyword_list = readCsv()

    for keyword in keyword_list[:5]:

        # 키워드 별로 담기 위해 리스트 초기화
        titleList = []

        # 페이지 수 지정하여 for문 실행
        for i in range(1, 10+1, 1):
            url = f"https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={i}"
            print(url)
            response = requests.get(url) # url 갖고오기
            html = response.text # text만 갖고오기
            soup = BeautifulSoup(html, 'html.parser') # 파싱하기

            titles = soup.select("ul.c-list-basic") # c-list-basic 클래스에서 ul에 묶여있는 요소들을 모두 갖고온다
            #subtitles = soup.select("a.elss.sub_tit")
            for title in titles : # titles에서 title   # 갖고온 요소로 for문 실행
                titleText = title.text # title.text만 추출
                titleList.append(titleText) # titleList에 추가
        
    return titleList

In [9]:
url_raw()

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=1
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=2
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=3
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=4
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=5
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=6
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=7
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=8
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=9
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=

['            한국일보       개별문서메뉴  톡으로 바로 공유 공유하기           4         [New & Good] 호텔 침대가 내 집으로 온다면? 일상을 휴일로 바꾸는 \'소노시즌\'       이어지도록 \'객실 쇼룸화\' 전략을 채택하고 있다. 소노시즌의 브랜드스토어가 소노캄(고양, 델피노), 쏠비치(양양, 삼척), 소노벨(천안, 비발디파크, 경주)에 위치한 것도 이런 이유에서다. 소노시즌은 서울과 인천에 총 5개의 직영점을 운영하고, 현대·롯데...    2024.04.25                 대전일보       개별문서메뉴  톡으로 바로 공유 공유하기           2         천안미술 싹 틔운 천안역 앞 다방들       밝히는 것은 미술관의 중요한 역할"이라며 "1990년대 아카이브 전시를 지속할 수 있도록 하겠다"고 말했다. 지난 19일 오후 소노벨천안에서 열린 \'천안미술 연구성과와 과제\' 학술세미나에서 최경현 천안시립미술관장이 기조강연을 하고 있다. 박하늘 기자 #충남 #천안    2024.04.21                 아시아경제       개별문서메뉴  톡으로 바로 공유 공유하기                 CJ프레시웨이, 컨세션 사업 3년간 年 36% 고성장…"워터파크 식음 서비스 강세"       식음 서비스를 운영하는 사업장은 ▲오션월드(강원 홍천) ▲스플라스 리솜(충남 예산) ▲웰리힐리파크(강원 횡성) ▲소노벨 천안(충남 천안) ▲포레스트 리솜(충북 제천) 등이다. CJ프레시웨이 관계자는 "식음 서비스를 이용하는 최종소비자에 그곳에서만 맛볼 수...    2024.04.25                 뉴시스       개별문서메뉴  톡으로 바로 공유 공유하기                 "푸드코트 시그니처 메뉴 개발" CJ프레시웨이, 컨세션 사업 3년간 36%씩 성장       식음 서비스를 운영하는 사업장은 오션월드(강원 홍천), 스플라스 리솜(충남 예산), 웰리힐리파크(강

In [36]:
def text_cleaning(text) :
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [37]:
# 3-1. 한글 뽑기

def get_hangul(titleList):
    
    # 한글만 추출하기
    titleList = list(map(lambda x:text_cleaning(x), titleList))
    #print(titleList)
    
    title_corpus = " ".join(titleList) # 추출한 각 요소를 띄어쓰기로 join
    nouns_tagger = Okt() # 형태소 초기화
    nouns = nouns_tagger.nouns(title_corpus) # 명사 추출
    
    count = Counter(nouns) # 명사 갯수 세기
    return count

In [38]:
# 3-2.불용어 필터링

def filter_hangul():
    # 불용어처리명
    remove_char_counter = Counter({x:count[x] for x in count if len(x) > 1})

    # 불용어 텍스트 파일 경로
    korean_stopwords_path = "data/stopwords-ko.txt"

    # 불용어 로드
    with open(korean_stopwords_path, encoding='utf8') as f:
        stopwords = f.readlines()
    stopwords = [x.strip() for x in stopwords]
    
    namu_wiki_stopwords = []

        # 불용어 추가
    stopwords.extend(namu_wiki_stopwords)

    # 불용어 처리
    remove_char_counter = Counter({x:remove_char_counter[x] for x in remove_char_counter if x not in stopwords})

    

In [39]:
# 3. 키워드 뽑기

def get_keyword():
    get_hangul(titleList):
    #불용어 필터링
    # 3-1.명사 나열 및 전처리(한글뽑기, 불용어 필터링)
    
    
    

SyntaxError: invalid syntax (895644777.py, line 4)

In [10]:
# main

main1 = url_raw()
get_keyword(main1)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=1
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=2
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=3
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=4
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=5
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=6
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=7
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=8
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=9
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=

In [19]:
get_hangul(main1)

Counter({'한국': 24,
         '일보': 8,
         '개별': 100,
         '문서': 100,
         '메뉴': 105,
         '톡': 100,
         '바로': 100,
         '공유': 204,
         '호텔': 52,
         '침대': 1,
         '내': 6,
         '집': 1,
         '일상': 1,
         '휴일': 1,
         '노': 123,
         '시즌': 24,
         '객실': 24,
         '쇼룸화': 1,
         '전략': 1,
         '채택': 1,
         '브랜드': 9,
         '스토어': 5,
         '노캄': 2,
         '고양': 11,
         '델피': 18,
         '쏠': 25,
         '비치': 26,
         '양양': 16,
         '삼척': 20,
         '노벨': 132,
         '천안': 166,
         '비발디': 20,
         '파크': 27,
         '경주': 6,
         '위치': 8,
         '것': 4,
         '이유': 2,
         '서울': 20,
         '인천': 1,
         '총': 6,
         '개': 34,
         '직영': 1,
         '점': 1,
         '운영': 36,
         '현대': 1,
         '롯데': 4,
         '대전': 7,
         '미술': 2,
         '싹': 1,
         '천안역': 1,
         '앞': 4,
         '다방': 1,
         '미술관': 2,
         '역할': 1,
